In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import options_trading as opt
%load_ext autoreload
%autoreload 2

In [ ]:
def down_and_in_put(S0, K, H, T, r, vol, N, M):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        BARRIER=False
        St=S0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
        
            if St<=H:
                BARRIER=True
        
        if BARRIER:
            CT=max(0,K-St)
        else:
            CT=0
            
        sum_CT=sum_CT+CT
        sum_CT2=sum_CT2+CT*CT
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)
    
    return C0

In [ ]:
def delta_gamma(S0, K, H, T, r, vol, N, M, epsilon):
    # Prix de l'option pour S0 + epsilon
    C_plus = down_and_in_put(S0 + epsilon, K, H, T, r, vol, N, M)
    
    # Prix de l'option pour S0 - epsilon
    C_minus = down_and_in_put(S0 - epsilon, K, H, T, r, vol, N, M)
    
    # Prix de l'option pour S0 (sans changement)
    C = down_and_in_put(S0, K, H, T, r, vol, N, M)
    
    # Calcul du delta
    delta = (C_plus - C_minus) / (2 * epsilon)
    
    # Calcul du gamma
    gamma = (C_plus - 2 * C + C_minus) / (epsilon ** 2)
    
    return delta, gamma

In [ ]:
def hedge_down_and_in(S0, K, H, T, r, vol, c, N, M):
    """
    c : cushion to manage risk around the barrier for hedging this type of option (int)
    """
    dt = T / N
    nudt = (r - 0.5 * vol**2) * dt
    volsdt = vol * np.sqrt(dt)
    erdt = np.exp(-r * dt)

    hedge_position_avg = np.zeros(N+1)
    cash_position_avg = np.zeros(N+1)
    PnL_avg = np.zeros(N+1)
    time = np.linspace(0, T, N+1)
    S = np.zeros(N+1)
    delta = np.zeros(N+1)

    for i in range(M):
        S[0] = S0
        hedge_position = np.zeros(N+1)
        cash_position = np.zeros(N+1)
        PnL = np.zeros(N+1)

        # Valeur initiale de l'option
        option_init_value = down_and_in_put(S[0], K, H - c, T, r, vol, N, M)
        
        # Calcule du delta initial
        delta[0], gamma = delta_gamma(S[0], K, H - c, T, r, vol, N, M, 0.01)
        
        # Position initiale de couverture
        hedge_position[0] = delta[0]
        
        # Calcul du PnL initial
        PnL[0] = hedge_position[0] * S[0] - option_init_value

        # Boucle temporelle
        for j in range(1, N + 1):
            epsilon = np.random.normal()
            Stn = S[j - 1] * np.exp(nudt + volsdt * epsilon)
            S[j] = Stn

            T_remaining = T - time[j]

            dS = S[j] - S[j - 1]

            # Recalcul du delta à chaque pas
            delta[j], new_gamma = delta_gamma(S[j], K, H - c, T_remaining, r, vol, N, M, dS)

            # Valeur actuelle de l'option
            option_value = down_and_in_put(S[j], K, H - c, T_remaining, r, vol, N, M)

            # Changement dans le delta
            delta_change = delta[j] - delta[j - 1]

            # Ajustement des positions de couverture
            hedge_position[j] = delta[j]

            # Mise à jour de la position en cash
            cash_position[j] = cash_position[j - 1] - delta_change * S[j]
            cash_position[j] *= np.exp(r * dt)  # Croissance du cash à taux r

            # Calcul du PnL
            PnL[j] = hedge_position[j] * S[j] + cash_position[j] - option_value

        # Moyenne des positions et du PnL
        hedge_position_avg += hedge_position
        cash_position_avg += cash_position
        PnL_avg += PnL

    # Moyenne finale sur M simulations
    hedge_position_avg /= M
    cash_position_avg /= M
    PnL_avg /= M

    return hedge_position_avg, cash_position_avg, PnL_avg, delta, time